# Análise de Marcha
**Bruno da Silva Cunha**

Universidade Federal do Espírio Santo

**1. Download, Crop e Gerar Keyframes**

Este notebook é responsável por realizar o download dos vídeos do youtube, fazer o crop, ou seja, pegar apenas a parte do vídeo que contém a marcha, e gerar os keyframes.

Neste notebook é importante utilizar uma instancia com GPU.

## Inicializando

In [2]:
!pip install yt_dlp
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive


Abaixo as constantes que serão utilizadas nesta seção

In [4]:
BASE_PATH = "/content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha"

As informações serão compartilhadas entre os Collabs pelo drive, então será necessário acessá-lo

In [5]:
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# Acessar a pasta onde contém todos os arquivos
%cd {BASE_PATH}
!ls

/content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha


## Carregamento dos dados

In [ ]:
import pandas as pd

In [ ]:
# Caminho onde estão os CSVs originais
DATA_PATH = BASE_PATH + "/data"
PROCESS_FILE_PATH = DATA_PATH + "/processed_gait_data.csv"
SAMPLE_RATE = 1 # Fins de desenvolvimento, usar uma taxa de amostragem para não realizar todos os processamentos
DEBUGG_RATE = 1 # Fins de desenvolvimento, essa taxa visa salvar uma parte dos vídeos / crops

In [ ]:
# Lê e concatena todos em um único DataFrame
df = pd.read_csv(PROCESS_FILE_PATH)

print(df.shape)
df.head()


(446, 12)


,seq,frame_num,cam_view,gait_pat,id,url,min_frame,max_frame,get_info_success,fps,start_time,end_time
0,cljvwb1h6001p3n6l138k9qo4,484,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,484,809,True,30.0,16.133,26.967
1,cljvwbtul001t3n6lu4qeic4a,889,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,889,1138,True,30.0,29.633,37.933
2,cljvwe0ky00213n6l7qczieno,5115,right side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,5115,5378,True,30.0,170.500,179.267
3,cljvwfmxb00293n6lyo0ffqwb,5761,right side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,5761,6148,True,30.0,192.033,204.933
4,cljvwitht002p3n6ltwn2u183,7103,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,7103,7348,True,30.0,236.767,244.933


In [ ]:
# Pegando uma amostra dos dados caso esteja configurado
if SAMPLE_RATE > 0 and SAMPLE_RATE < 1:
  sampled_ids = df['id'].drop_duplicates().sample(frac=SAMPLE_RATE)
  df = df[df['id'].isin(sampled_ids)]
  print(sampled_ids.info())

In [ ]:
  df


,seq,frame_num,cam_view,gait_pat,id,url,min_frame,max_frame,get_info_success,fps,start_time,end_time
0,cljvwb1h6001p3n6l138k9qo4,484,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,484,809,True,30.0,16.133,26.967
1,cljvwbtul001t3n6lu4qeic4a,889,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,889,1138,True,30.0,29.633,37.933
2,cljvwe0ky00213n6l7qczieno,5115,right side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,5115,5378,True,30.0,170.500,179.267
3,cljvwfmxb00293n6lyo0ffqwb,5761,right side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,5761,6148,True,30.0,192.033,204.933
4,cljvwitht002p3n6ltwn2u183,7103,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,7103,7348,True,30.0,236.767,244.933
...,...,...,...,...,...,...,...,...,...,...,...,...
441,cljr6wkld00193n6ltjloqzl1,469,left side,myopathic,9qg-3smwLcs,https://www.youtube.com/watch?v=9qg-3smwLcs,469,732,True,30.0,15.633,24.400
442,cljr6xqvi001d3n6lctvtihw2,736,right side,myopathic,9qg-3smwLcs,https://www.youtube.com/watch?v=9qg-3smwLcs,736,1190,True,30.0,24.533,39.667
443,cljr6z0uo001h3n6lixdz25qa,1193,left side,myopathic,9qg-3smwLcs,https://www.youtube.com/watch?v=9qg-3smwLcs,1193,1439,True,30.0,39.767,47.967
444,cljr6zqwu001l3n6lc40rmemx,1486,left side,myopathic,9qg-3smwLcs,https://www.youtube.com/watch?v=9qg-3smwLcs,1486,1907,True,30.0,49.533,63.567


In [ ]:
print("Quantidade de vídeos:", len(df.groupby("id")))

Quantidade de vídeos: 69


## Download dos vídeos

Nessa etapa é realizado o download dos vídeos do youtube para que sejam deitos as análises. Pode ser configurado uma porcentagem que os vídeos serão salvos para debugger.

In [ ]:
from pathlib import Path
import yt_dlp
from tqdm import tqdm
from IPython.display import Video
import time




ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-3238952555.py", line 2, in <cell line: 0>
    import yt_dlp
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1322, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 1262, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1532, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1504, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1483, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
   

In [ ]:
BASE_DIR = Path(BASE_PATH)
PROCESSED_DIR = Path(BASE_DIR / f"processed"); PROCESSED_DIR.mkdir(exist_ok=True, parents=True)
VIDEOS_DIR = Path(PROCESSED_DIR / f"videos"); VIDEOS_DIR.mkdir(exist_ok=True, parents=True)


In [ ]:
def is_video_downloaded(id, path):
  path = path / f"{id}_video.mp4"
  return path.exists()

# Função que realiza o download do vídeo e salva no caminho especificado
def download_video(id, row, path):
  out = path / f"{id}_video.mp4"
  # Caso o vídeos já esteja salvo, não realiza o download novamente
  if is_video_downloaded(id, path):
    return out

  url = row['url'] if pd.notna(row.get('url')) else f"https://www.youtube.com/watch?v={id}"
  out = path / f"{id}_video.mp4"
  ydl_opts = {
    "outtmpl": str(out),
    "format": "bestvideo[height<=720][ext=mp4]",
    "merge_output_format": "mp4",
    "cookiefile": "/content/cookies.txt"
  }
  try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      ydl.extract_info(url, download=True)
  except Exception as e:
    print(f"Erro ao baixar o vídeo {id}: {e}")
    return None



In [ ]:
if "download_video_success" not in df.columns:
  df["download_video_success"] = None

# Reseta a coluna download_vide_success de false para None, para reprocessar
df.loc[df["download_video_success"] == False, "download_video_success"] = None


# Percorre todo o dataframe e realiza o download dos vídeos
for id, row in tqdm(df.iterrows(), total=len(df)):
  if df.loc[df["id"] == row["id"], "download_video_success"].iloc[0] is not None:
    continue

  path = download_video(row["id"], row, VIDEOS_DIR)
  if path is None:
    df.loc[df["id"] == row["id"], "download_video_success"] = False
  else:
    df.loc[df["id"] == row["id"], "download_video_success"] = True
    df.loc[df["id"] == row["id"], "video_path"] = path
  time.sleep(1)

df.head()

  0%|          | 0/446 [00:00<?, ?it/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=338R10fA_wU
[youtube] 338R10fA_wU: Downloading webpage
[youtube] 338R10fA_wU: Downloading tv client config
[youtube] 338R10fA_wU: Downloading tv player API JSON
[youtube] 338R10fA_wU: Downloading web safari player API JSON
[youtube] 338R10fA_wU: Downloading m3u8 information
[info] 338R10fA_wU: Downloading 1 format(s): 136
[download] Sleeping 4.00 seconds as required by the site...
[download] Resuming download at byte 14663989
[download] Destination: /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/processed/videos/crops/338R10fA_wU_video.mp4
[download] 100% of   53.36MiB in 00:00:17 at 3.07MiB/s   


  0%|          | 1/446 [00:25<3:10:55, 25.74s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=9126Bdjn8sg
[youtube] 9126Bdjn8sg: Downloading webpage
[youtube] 9126Bdjn8sg: Downloading tv client config
[youtube] 9126Bdjn8sg: Downloading tv player API JSON
[youtube] 9126Bdjn8sg: Downloading web safari player API JSON
[youtube] 9126Bdjn8sg: Downloading m3u8 information
[info] 9126Bdjn8sg: Downloading 1 format(s): 136
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/processed/videos/crops/9126Bdjn8sg_video.mp4
[download] 100% of   51.64MiB in 00:00:04 at 12.88MiB/s  


 13%|█▎        | 56/446 [00:39<02:42,  2.40it/s]

...

[youtube] Extracting URL: https://www.youtube.com/watch?v=7nIQv3K_-Qc
[youtube] 7nIQv3K_-Qc: Downloading webpage
[youtube] 7nIQv3K_-Qc: Downloading tv client config
[youtube] 7nIQv3K_-Qc: Downloading tv player API JSON
[youtube] 7nIQv3K_-Qc: Downloading web safari player API JSON
[youtube] 7nIQv3K_-Qc: Downloading m3u8 information
[info] 7nIQv3K_-Qc: Downloading 1 format(s): 136
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/processed/videos/crops/7nIQv3K_-Qc_video.mp4
[download] 100% of   18.32MiB in 00:00:02 at 7.29MiB/s   


 93%|█████████▎| 417/446 [15:01<01:13,  2.54s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=8mTHlAIdea0
[youtube] 8mTHlAIdea0: Downloading webpage
[youtube] 8mTHlAIdea0: Downloading tv client config
[youtube] 8mTHlAIdea0: Downloading tv player API JSON
[youtube] 8mTHlAIdea0: Downloading web safari player API JSON
[youtube] 8mTHlAIdea0: Downloading m3u8 information
[info] 8mTHlAIdea0: Downloading 1 format(s): 136
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: /content/drive/MyDrive/Estudos/IA_CD/Modelo_Treino_Marcha/processed/videos/crops/8mTHlAIdea0_video.mp4
[download] 100% of   16.03MiB in 00:00:05 at 3.15MiB/s   


100%|██████████| 446/446 [15:16<00:00,  2.06s/it]


,seq,frame_num,cam_view,gait_pat,id,url,min_frame,max_frame,get_info_success,fps,start_time,end_time,download_video_success,video_path
0,cljvwb1h6001p3n6l138k9qo4,484,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,484,809,True,30.0,16.133,26.967,False,NaN
1,cljvwbtul001t3n6lu4qeic4a,889,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,889,1138,True,30.0,29.633,37.933,False,NaN
2,cljvwe0ky00213n6l7qczieno,5115,right side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,5115,5378,True,30.0,170.500,179.267,False,NaN
3,cljvwfmxb00293n6lyo0ffqwb,5761,right side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,5761,6148,True,30.0,192.033,204.933,False,NaN
4,cljvwitht002p3n6ltwn2u183,7103,left side,cerebral palsy,338R10fA_wU,https://www.youtube.com/watch?v=338R10fA_wU,7103,7348,True,30.0,236.767,244.933,False,NaN


Verificar se todos os vídeos foram baixados com sucesso. Caso contrário, executar o bloco acima novamente

In [ ]:
df.value_counts('download_video_success')

,count
download_video_success,
False,346
True,100


In [ ]:
Video(df.iloc[0]["video_path"], embed=True, height=300)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not float

## Cropar vídeos


Neste etapa será feito o corte dos vídeos para pegar apenas os momentos que ocorrem a marcha

In [ ]:
import subprocess
import torch



In [ ]:
import subprocess, shlex, os, tempfile
from pathlib import Path

# Garante que VIDEOS_DIR exista
VIDEOS_DIR = Path(PROCESSED_DIR / f"videos/crops"); VIDEOS_DIR.mkdir(exist_ok=True, parents=True)

def run_ffmpeg(cmd):
    # cmd é uma lista; executa e lança erro se falhar
    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if proc.returncode != 0:
        raise RuntimeError(f"FFmpeg falhou:\n{proc.stderr}")
    return proc

def cut_segment(input_path, start_time, end_time, fps, use_nvenc=False):
    """Corta um trecho e retorna o caminho do arquivo temporário gerado.
       precise=True => reencode para cortes exatos
    """
    import tempfile, os, subprocess
    tmp_fd, tmp_path = tempfile.mkstemp(suffix=".mp4")
    os.close(tmp_fd)
    tmp_path = Path(tmp_path)

    # ordem: -ss antes do -i -> corte rápido, menos preciso
    cmd = [
        "ffmpeg",
        "-y",
        "-ss", str(start_time),
        "-to", str(end_time),
        "-i", str(input_path),
        "-c", "copy"
    ]

    cmd += [str(tmp_path)]

    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if proc.returncode != 0:
        raise RuntimeError(f"Erro ao cortar {input_path}:\n{proc.stderr}")
    return tmp_path


def concat_segments(segment_paths, out_path):
    """Concatena segmentos usando concat demuxer (sem recompactar áudio entre cortes)."""
    # Cria lista para o demuxer
    with tempfile.NamedTemporaryFile("w", delete=False, suffix=".txt") as f:
        list_path = f.name
        for p in segment_paths:
            # Escapa corretamente eventuais aspas simples e espaços
            safe_path = str(p).replace("'", "'\\''")
            f.write(f"file '{safe_path}'\n")

    # Para evitar erros de timebase, fazemos uma recompressão única na concatenação
    cmd = [
        "ffmpeg",
        "-y",
        "-f", "concat",
        "-safe", "0",
        "-i", list_path,
        "-c:v", "copy",  # tenta copiar; se der problema, reencode abaixo (descomente linha)
        "-c:a", "aac", "-b:a", "128k",
        str(out_path)
    ]
    # Se copiar falhar devido a parâmetros distintos, troque "-c:v copy" por libx264/h264_nvenc como no corte.

    try:
        run_ffmpeg(cmd)
    finally:
        try:
            os.remove(list_path)
        except OSError:
            pass

def cortar_e_manter_partes(df):
    """
    Para cada (id, video_path), corta as janelas [start_time, end_time] e concatena,
    gerando um único arquivo {id}_crop.mp4 em VIDEOS_DIR.
    Atualiza df['crop_path'] para todas as linhas do grupo.
    """
    use_nvenc = torch.cuda.is_available()

    # Ordena por início para concatenar em ordem
    df_sorted = df.sort_values(["id", "video_path", "start_time"]).copy()

    for (vid, vpath, fps), group in df_sorted.groupby(["id", "video_path", "fps"], sort=False):
        input_path = Path(vpath)
        out_path = VIDEOS_DIR / f"{vid}_crop.mp4"

        if out_path.exists():
            print(f"✅ {out_path.name} já existe — pulando processamento.")
            df.loc[group.index, "crop_path"] = str(out_path)
            continue

        print(f"\nProcessando vídeo {vid} -> {input_path}")

        # 1) corta janelas
        segments = []
        try:
            for _, row in group.iterrows():
                s, e = float(row["start_time"]), float(row["end_time"])
                print(f"  - Cortando janela [{s:.3f}s, {e:.3f}s]")
                seg = cut_segment(input_path, s, e, fps, use_nvenc=use_nvenc)
                segments.append(seg)

            # 2) concatena
            print(f"  - Concatenando {len(segments)} segmentos em {out_path}")
            concat_segments(segments, out_path)

            # 3) atualiza df para todas as linhas deste grupo
            idxs = group.index
            df.loc[idxs, "crop_path"] = str(out_path)

        finally:
            # Remove temporários
            for p in segments:
                try:
                    Path(p).unlink(missing_ok=True)
                except Exception:
                    pass

    return df


In [ ]:
df = cortar_e_manter_partes(df)
print("Concluído.")

In [ ]:
unique_ids = df['id'].drop_duplicates().tolist()
for video_id in unique_ids[:5]:
    video_path = df[df['id'] == video_id]['crop_path'].iloc[0]
    print(f"Displaying video for ID: {video_id}")
    display(Video(video_path, embed=True, height=300))

## Gerar os Keyframes

Nesta etapa pegará todos os vídeos e será utilizado o YOLO para gerar os keyframes

In [ ]:
from scipy.signal import savgol_filter
from ultralytics import YOLO
import numpy as np
import random


In [ ]:
def clean_kps_seq(kps_seq):
    """
    Limpa e normaliza uma sequência de keypoints [T,17,3]
    Retorna uma nova sequência com (x,y) centralizados e normalizados.
    """
    kps = kps_seq.copy()

    # 1. Substituir NaNs por interpolação linear
    for j in range(kps.shape[1]*2):  # x e y de cada ponto
        t_series = kps[:, j//2, j%2]
        nans = np.isnan(t_series)
        if nans.any():
            idx = np.arange(len(t_series))
            if (~nans).sum() > 1:
                t_series[nans] = np.interp(idx[nans], idx[~nans], t_series[~nans])
            else:
                t_series[nans] = 0
        kps[:, j//2, j%2] = t_series

    # 2. Zerar keypoints com confiança muito baixa
    conf_mask = kps[:,:,2] < 0.2
    kps[conf_mask, :2] = np.nan

    # 3. Centralizar no quadril (média dos keypoints 11 e 12 - padrão COCO)
    left_hip, right_hip = 11, 12
    center = np.nanmean(kps[:, [left_hip, right_hip], :2], axis=1)
    kps[:, :, :2] -= center[:, np.newaxis, :]

    # 4. Normalizar pela distância entre ombros (escala corporal)
    left_sh, right_sh = 5, 6
    shoulder_dist = np.linalg.norm(kps[:, left_sh, :2] - kps[:, right_sh, :2], axis=1)
    scale = np.nanmedian(shoulder_dist)
    kps[:, :, :2] /= (scale + 1e-6)

    # 5. Suavizar as trajetórias
    for j in range(kps.shape[1]*2):
        seq = kps[:, j//2, j%2]
        if len(seq) >= 9:
            kps[:, j//2, j%2] = savgol_filter(seq, window_length=9, polyorder=2)

    return kps

In [1]:
# --- config ---
CONF = 0.25
IMG_SIZE = 640
VID_STRIDE = 2          # processa 1 a cada 2 frames (mais rápido)
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

# parâmetros anti-falso-positivo
MIN_MULTI_SECONDS = 0.4  # tempo mínimo contínuo com >1 pessoa para descartar
DEFAULT_FPS = 30.0       # fallback se df não tiver fps

# pastas
POSES_DIR = Path(PROCESSED_DIR) / "poses"
POSES_DIR.mkdir(parents=True, exist_ok=True)

# cols obrigatórias
for col in ("multi_person", "poses_path"):
    if col not in df.columns:
        df[col] = None if col == "poses_path" else False

# modelo
model = YOLO("yolo11n-pose.pt")

# antes do loop
df_uniques = df.sort_values(by=["id"]).drop_duplicates(subset=["id"], keep="first").reset_index(drop=True)

for idx, row in tqdm(df_uniques.iterrows(), total=len(df_uniques)):
    vid_id   = row["id"]
    src_path = Path(row.get("crop_path") or row.get("video_path"))
    out_path = POSES_DIR / f"{vid_id}_poses.npy"

    # pular se já existe npy
    if out_path.exists():
        df_uniques.loc[df_uniques["id"] == vid_id, "poses_path"] = str(out_path)
        continue

    # pular se já foi detectado multiperson
    if df_uniques.loc[df_uniques["id"] == vid_id, "multi_person"].iloc[0] == True:
        continue

    print(f"▶️  Processando {vid_id} -> {src_path}")

     # --- novo: calcula frames mínimos com base no fps/tempo
    fps = float(row["fps"]) if "fps" in df_uniques.columns else DEFAULT_FPS
    min_multi_frames = max(2, int(round(fps * MIN_MULTI_SECONDS)))  # pelo menos 2 frames
    multi_consec = 0
    multi_flag = False

    # debug visual aleatório
    save_vis = (random.random() < DEBUGG_RATE)

    # stream = True para iterar frame a frame
    preds = model.predict(
        source=str(src_path),
        stream=True,
        conf=CONF,
        imgsz=IMG_SIZE,
        vid_stride=VID_STRIDE,
        device=DEVICE,
        verbose=False,
        save=save_vis,
        exist_ok=True,
        project=VIDEOS_DIR
    )

    kps_seq = []

    for r in preds:
        # conta detecções no frame
        n_people = len(r.boxes) if (r.boxes is not None) else 0

        # --- novo: exige sequência mínima de frames com >1 pessoa
        if n_people > 1:
            multi_consec += 1
            if multi_consec >= min_multi_frames:
                multi_flag = True
                print(f"❌ >1 pessoa por {multi_consec} frames (~{multi_consec/fps:.2f}s) — descartando: {vid_id}")
                break
        else:
            multi_consec = 0  # reset se voltou a <=1

        # nenhum keypoint no frame → preenche com NaN (mantém alinhamento temporal)
        if (r.keypoints is None) or (len(r.keypoints) == 0):
            kps_seq.append(np.full((17, 3), np.nan, dtype=np.float32))
            continue

        # escolhe a pessoa principal (maior bbox)
        areas = (r.boxes.xywh[:, 2] * r.boxes.xywh[:, 3]).cpu().numpy()
        i = int(areas.argmax())
        kps = r.keypoints.data[i].cpu().numpy()  # (17,3)
        kps_seq.append(kps.astype(np.float32))

    if multi_flag:
        df_uniques.loc[df_uniques["id"] == vid_id, "multi_person"] = True
        ## remove o video
        if out_path.exists():
            try: os.remove(out_path)
            except: pass
        continue

    # se não gerou nada, pula
    if len(kps_seq) == 0:
        print(f"⚠️  Sem frames válidos: {vid_id}")
        continue

    # limpeza e salvamento
    kps_seq = np.asarray(kps_seq, dtype=np.float32)   # [T,17,3]
    kps_seq_clean = clean_kps_seq(kps_seq)            # sua função
    np.save(out_path, kps_seq_clean)

    df_uniques.loc[df_uniques["id"] == vid_id, "poses_path"] = str(out_path)


In [ ]:
input_pose = VIDEOS_DIR / f'predict/{df_uniques.iloc[0].id}_crop.avi'
output_pose = VIDEOS_DIR / f'predict/{df_uniques.iloc[0].id}_pose.mp4'
cmd = [
  "ffmpeg",
  "-i", input_pose,
  "-vcodec", "libx264",
  output_pose,
]
subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
Video(output_pose, embed=True, height=300)

In [ ]:
df_uniques[df_uniques["multi_person"] == True]

In [ ]:
before = len(df_uniques)
cleaned = df_uniques[df_uniques["multi_person"] != True].copy()
after = len(cleaned)

print(f"🧹 Removidos {before - after} vídeos com múltiplas pessoas. Restam {after}.")

In [ ]:
cleaned